In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from sklearn.metrics import f1_score
from os import path
from os import listdir
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch
from math import log2, e
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Mounted at /content/drive


In [ ]:
f = open("drive/MyDrive/nietzsche.txt", "r")
n = 517500
data = [0] * n
s = set()
for i in range(n):
  words = f.readline().strip()
  words = words.lower()
  data[i] = words[:-1]
  for x in words:
    s.add(x)
print(s)

{'k', '.', 'e', 'a', 'o', 't', ':', '!', 'j', "'", '0', 'n', 'p', '=', 'q', '"', 'æ', ' ', '3', '-', 'w', ')', 'd', ']', 'i', 'y', 'h', '7', 'l', '_', '6', '9', 'r', 's', ',', 'u', 'v', '5', '?', 'c', '4', '[', 'g', 'f', 'm', ';', '(', 'z', 'b', '8', '1', '2', 'x', 'ë', 'é', 'ä'}


In [ ]:
todel = "_~,£;)=\"?*(/\'$!-&+#—.:%=\[\]"
for c in todel:
  for i in range(len(data)):
    data[i] = data[i].replace(c, "")
s = set()
for x in data:
  for symb in x:
    s.add(symb)
s = (list(s))
s.sort()
print(s)

[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']


In [ ]:
index = 0
symb_to_int = dict()
int_to_symb = dict()
for c in s:
  if (c not in symb_to_int):
    symb_to_int[c] = index
    int_to_symb[index] = c
    index += 1

In [ ]:
def to_num(seq):
  ans = []
  for j in seq:
    ans.append(symb_to_int[j])
  return ans

real_data = [0] * len(data)
for i in range(len(data)):
  real_data[i] = to_num(data[i])
print(real_data[7])
print(data[7])

[30, 18, 15, 19, 28, 0, 11, 14, 14, 28, 15, 29, 29, 15, 29, 0, 30, 25, 0, 30, 28, 31, 30, 18, 0, 18, 11, 32, 15, 0, 12, 15, 15, 24, 0, 31, 24, 29, 21, 19, 22, 22, 15, 14, 0, 11, 24, 14, 0, 31, 24, 29, 15, 15, 23, 22, 35, 0, 23, 15, 30, 18, 25, 14, 29, 0, 16, 25]
their addresses to truth have been unskilled and unseemly methods fo


In [ ]:
print(data[0:20])

['prefac', '', '', 'supposing that truth is a womanwhat then is there not groun', 'for suspecting that all philosophers in so far as they have bee', 'dogmatists have failed to understand womenthat the terribl', 'seriousness and clumsy importunity with which they have usually pai', 'their addresses to truth have been unskilled and unseemly methods fo', 'winning a woman certainly she has never allowed herself to be won an', 'at present every kind of dogma stands with sad and discouraged mienif', 'indeed it stands at all for there are scoffers who maintain that i', 'has fallen that all dogma lies on the groundnay more that it is a', 'its last gasp but to speak seriously there are good grounds for hopin', 'that all dogmatizing in philosophy whatever solemn whatever conclusiv', 'and decided airs it has assumed may have been only a noble puerilis', 'and tyronism and probably the time is at hand when it will be onc', 'and again understood what has actually sufficed for the basis of suc', 'imp

In [ ]:
seq = 20
X_Y = []
for i in range(len(data)):
  for j in range(len(data[i])):
    if (j + seq + 1 < len(data[i])):
      X_Y.append((real_data[i][j:j + seq], real_data[i][j + 1:j + seq + 1]))
print(X_Y[0])

([29, 31, 26, 26, 25, 29, 19, 24, 17, 0, 30, 18, 11, 30, 0, 30, 28, 31, 30, 18], [31, 26, 26, 25, 29, 19, 24, 17, 0, 30, 18, 11, 30, 0, 30, 28, 31, 30, 18, 0])


In [ ]:
class LSTM(torch.nn.Module):
  def __init__(self):
    super(LSTM, self).__init__()
    self.features = len(s)
    self.embedd = self.features
    self.layers = 1
    self.embedding = torch.nn.Embedding(
        num_embeddings=self.features,
        embedding_dim=self.embedd,
    )
    self.lstm = torch.nn.LSTM(
        input_size=self.embedd,
        hidden_size=self.embedd,
        num_layers=self.layers
    )
    self.fc = torch.nn.Linear(self.embedd, len(s))
  def forward(self, x, prev_state):
    embed = self.embedding(x)
    output, state = self.lstm(embed, prev_state)
    logits = self.fc(output)
    return logits, state

  def init_state(self):
    return (torch.zeros(self.layers, seq, self.embedd), 
            torch.zeros(self.layers, seq, self.embedd))

class Dataset(torch.utils.data.Dataset):
  def __init__(self, data):
    self.data = data
      
  def __len__(self):
    return len(self.data)

  def __getitem__(self, i):
    return (torch.LongTensor(self.data[i][0]), torch.LongTensor(self.data[i][1]))

In [ ]:
model = LSTM().to(device)
batch_size = 1000

In [ ]:
def train(model):
  dataset = Dataset(X_Y)
  dataloader = DataLoader(dataset, batch_size, num_workers=6)
  model.train()
  for _, (x, y) in tqdm(enumerate(dataloader)):
    h, c = model.init_state()
    optimizer.zero_grad()
    x = x.to(device)
    y = y.to(device)
    h = h.to(device)
    c = c.to(device)
    y_pred, (h, c) = model(x, (h, c))
    
    loss = criterion(y_pred.transpose(1, 2), y)
    loss.backward()
    optimizer.step()
    print(loss.item())

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train(model)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
3it [00:00,  9.96it/s]

3.6712663173675537
3.658856153488159
3.6463820934295654
3.6367902755737305


9it [00:00, 18.38it/s]

3.6318278312683105
3.619499683380127
3.60658597946167
3.599594831466675
3.574824571609497
3.562448024749756


15it [00:00, 22.71it/s]

...

2.2048962116241455
2.17576265335083
2.219677448272705
2.238610029220581
2.2575132846832275
2.2277328968048096
2.2037017345428467


374it [00:14, 29.84it/s]

2.223416566848755
2.22510027885437
2.278574228286743
2.2098472118377686
2.238264560699463
2.154798984527588
2.212110996246338


378it [00:14, 25.32it/s]

2.2500739097595215
2.1174514293670654


In [ ]:
def predict(pref, model, amount):
  model.eval()
  ans = pref
  tens = []
  for j in pref:
    tens.append(symb_to_int[j])
  h, c = model.init_state()
  while (len(ans) < amount): 
    x = torch.tensor([tens[-seq:]])
    x = x.to(device)
    h = h.to(device)
    c = c.to(device)
    y_pred, (h, c) = model(x, (h, c))
    p = torch.nn.functional.softmax(y_pred[0][-1], dim=0).cpu().detach().numpy()
    index = np.random.choice(len(y_pred[0][-1]), p=p)
    tens.append(index)
    ans += int_to_symb[index]

  return ans

In [ ]:
for i in range(4):
  print(predict('during this process ', model, 777))

during this process condes an bosurengionces th im ir nuxent theg int thtaro whema weloicina to anlof whe dostonsird toanth moqud undis okhe hetdunro los in1erd for is tcecenlo tougias euntin the wo auos eocth in ang cigosud doth was schof madion agir wiy ilsr orsis oifang froancy delssie waty thit antlo aly thel tive meot nicate wher con omatheteko lhe b tto 1osta tion id erppiesthe thed if thif io l averipes pha mollmughas skon esco a ind withe el cuthe reo thery coungmy the nese the e dige theed coreslemf d imcimrect rofi5 the anur at oburin ell fands emy enufled onæe ting bamof othitl as oct eawt toesry caidto modin ib peresus as theif ont cythe sate al as seneves ojt astiogns scobanthigond arin sighanat galid the an mege ans tiin tod feannes bapporeiotes urdceun
during this process äkore of stis hen macth thitle sal whiontetttsis at harinopilleee nef mesugh p6imy cheven inerve tul id ingresl rarid erwt path thels sof nd whon abaratle etie that ocug5ragt ales theste aso ileess augl

In [ ]:
class MARK:
  def __init__(self, sz):
    self.autom = dict()
    self.autom_sz = dict()
    self.size = sz

  def train(self, text):
    for i in range(len(text)):
      if (i + self.size >= len(text)):
        break
      key = text[i:i + self.size]
      symb = text[i + self.size]
      if (key in self.autom):
        self.autom[key][symb_to_int[symb]] += 1
        self.autom_sz[key] += 1
      else:
        self.autom[key] = np.zeros(len(s))
        self.autom[key][symb_to_int[symb]] += 1
        self.autom_sz[key] = 1

  def predict(self, pref, size):
    ans = pref
    prob = dict()
    for key in self.autom:
      prob[key] = np.zeros(len(s))
      for j in range(len(s)):
        if (self.autom_sz[key] > 0):
          prob[key][j] = self.autom[key][j] / self.autom_sz[key]
      #print(prob[key])
    while (len(ans) < size):
      key = ans[-self.size:]
      next = np.random.choice(len(s), p=prob[key])
      ans += int_to_symb[next]
    return ans

mark = MARK(4)

In [ ]:
for dat in data:
  mark.train(dat)

In [ ]:
for i in range(3):
  print(mark.predict("during this process", 100))

during this processes have individual independed the but a pride inded displace god head strikes hap
during this processandfather justic righty upon some parent trict earn arranged to be ever conding o
during this process too what what is always only by mean no the suffering for in true decret accurat
